In [11]:
from scamp import *

# Session will be created fresh in each playback cell to avoid clock drift

In [12]:
import math
from types import SimpleNamespace
import time

# --- CONSTANTS ---

# Western music has 12 notes per octave; 's' = sharp (# isn't valid in identifiers)
CHROMATIC_SCALE = ['C', 'Cs', 'D', 'Ds', 'E', 'F', 'Fs', 'G', 'Gs', 'A', 'As', 'B']
NOTES_PER_OCTAVE = len(CHROMATIC_SCALE)

# Flats are alternate names for sharps (Db and Cs are the same pitch)
FLAT_TO_SHARP = {'Db': 'Cs', 'Eb': 'Ds', 'Gb': 'Fs', 'Ab': 'Gs', 'Bb': 'As'}

# MIDI uses 0-127; octave -1 is lowest, 9 is highest
MIDI_MIN = 0
MIDI_MAX = 127
LOWEST_OCTAVE = -1
HIGHEST_OCTAVE = 9
NEGATIVE_OCTAVE_LABEL = 'm1'  # Python identifiers can't have minus signs

# Frequency-to-MIDI conversion reference point (A4 = 440 Hz = MIDI 69)
A4_FREQUENCY_HZ = 440
A4_MIDI_NUMBER = 69


# --- MAIN ---

def build_pitches():
    """
    Build namespace of all MIDI pitches. Access via N.C4, N.Fs4, N.Bb3, etc.
    SimpleNamespace lets us use N.C4 (dot notation) instead of N['C4'].
    """
    pitches = {}
    for octave in range(LOWEST_OCTAVE, HIGHEST_OCTAVE + 1):
        add_pitches_for_octave(pitches, octave)
        add_flat_aliases_for_octave(pitches, octave)
    return SimpleNamespace(**pitches)


def add_pitches_for_octave(pitches, octave):
    """Add all 12 notes for a single octave."""
    for position, note_name in enumerate(CHROMATIC_SCALE):
        try_add_pitch(pitches, note_name, octave, position)


def add_flat_aliases_for_octave(pitches, octave):
    """Add flat aliases (Db, Eb, etc.) for a single octave."""
    for flat, sharp in FLAT_TO_SHARP.items():
        try_add_flat_alias(pitches, flat, sharp, octave)

def try_add_pitch(pitches, note_name, octave, position):
    """Add a single pitch to dict if MIDI value is valid."""
    midi = note_to_midi(octave, position)
    if is_valid_midi(midi):
        pitches[make_note_key(note_name, octave)] = midi


def try_add_flat_alias(pitches, flat, sharp, octave):
    """Add a flat alias if the corresponding sharp exists."""
    sharp_key = make_note_key(sharp, octave)
    if sharp_key in pitches:
        pitches[make_note_key(flat, octave)] = pitches[sharp_key]


def make_note_key(note_name, octave):
    """Create key like 'C4' or 'Fsm1'."""
    return f'{note_name}{octave_to_label(octave)}'


def note_to_midi(octave, position):
    """MIDI number = (octave + 1) * 12 + position. C4 = 60."""
    return (octave + 1) * NOTES_PER_OCTAVE + position


def is_valid_midi(midi_number):
    return MIDI_MIN <= midi_number <= MIDI_MAX


def octave_to_label(octave):
    """Convert octave number to valid identifier suffix ('-1' becomes 'm1')."""
    return NEGATIVE_OCTAVE_LABEL if octave == LOWEST_OCTAVE else str(octave)


def hz_to_midi(freq):
    """Convert frequency (Hz) to MIDI note number. A4 (440 Hz) = 69."""
    return A4_MIDI_NUMBER + NOTES_PER_OCTAVE * math.log2(freq / A4_FREQUENCY_HZ)


# --- CREATE NOTE LOOKUP ---

N = build_pitches()

print(f"N.C4={N.C4} (middle C), N.A4={N.A4} (440Hz), N.Fs4={N.Fs4}, N.Gb4={N.Gb4}")


N.C4=60 (middle C), N.A4=69 (440Hz), N.Fs4=66, N.Gb4=66


In [13]:
# play_note(pitch, volume, duration)
#   pitch:    MIDI note number (0-127), or fractional for microtones
#   volume:   Loudness from 0.0 (silent) to 1.0 (max)
#   duration: Length in beats (at the Session tempo, e.g. 100 BPM)

# Fresh session each run — avoids clock drift on re-runs
s = Session(tempo=100)
piano = s.new_part("piano")

# Playback settings
VOLUME = 0.8
SHORT = 0.5   # half beat
LONG = 1.0    # full beat

# --- EXAMPLES ---

# 1. Lowest possible note (MIDI 0 = C in octave -1, ~8 Hz — may be inaudible!)
print(f"Lowest note: N.Cm1 = {N.Cm1}")
piano.play_note(N.Cm1, VOLUME, SHORT)

# 2. Highest possible note (MIDI 127 = G9, ~12,544 Hz — very high pitched!)
print(f"Highest note: N.G9 = {N.G9}")
piano.play_note(N.G9, VOLUME, SHORT)

# 3. Play a note by frequency (Hz) — 261.63 Hz is middle C
freq = 261.63
midi_from_freq = hz_to_midi(freq)
print(f"Playing {freq} Hz = MIDI {midi_from_freq:.1f}")
piano.play_note(midi_from_freq, VOLUME, SHORT)

# 4. C major arpeggio
piano.play_note(N.C4, VOLUME, SHORT)
piano.play_note(N.E4, VOLUME, SHORT)
piano.play_note(N.G4, VOLUME, SHORT)
piano.play_note(N.C5, VOLUME, LONG)

print("Done!")


Using preset Piano Merlin for piano
Lowest note: N.Cm1 = 0
Highest note: N.G9 = 127
Playing 261.63 Hz = MIDI 60.0
Done!


In [17]:
# --- CHORDS ---
# A chord is multiple notes played simultaneously

# Fresh session
s = Session(tempo=100)
piano = s.new_part("piano")

# Playback settings
VOLUME = 0.8
CHORD_DURATION = 1.0

def play_chord(instrument, notes, volume, duration):
    """Play multiple notes simultaneously as a chord."""
    # Play all notes without blocking (they start together)
    for note in notes[:-1]:
        instrument.play_note(note, volume, duration, blocking=False)
    # Last note blocks so we wait for the chord duration
    instrument.play_note(notes[-1], volume, duration, blocking=True)


# Define some common chords (root position triads)
C_MAJOR = [N.C4, N.E4, N.G4]      # C-E-G
G_MAJOR = [N.G3, N.B3, N.D4]      # G-B-D
A_MINOR = [N.A3, N.C4, N.E4]      # A-C-E
F_MAJOR = [N.F3, N.A3, N.C4]      # F-A-C

# Play a I-V-vi-IV progression (most common pop progression)
print("Playing I-V-vi-IV progression in C major...")
play_chord(piano, C_MAJOR, VOLUME, CHORD_DURATION)   # I  (C)
play_chord(piano, G_MAJOR, VOLUME, CHORD_DURATION)   # V  (G)
play_chord(piano, A_MINOR, VOLUME, CHORD_DURATION)   # vi (Am)
play_chord(piano, F_MAJOR, VOLUME, CHORD_DURATION)   # IV (F)

print("Done!")


Using preset Piano Merlin for piano
Playing I-V-vi-IV progression in C major...
Done!


In [15]:
# --- DRUM BEAT ---
# Classic pop/rock pattern: kick on 1 & 3, snare on 2 & 4, hi-hat on eighth notes

# General MIDI drum note numbers (channel 10 standard)
KICK = 36       # Bass drum
SNARE = 38      # Acoustic snare
HIHAT = 42      # Closed hi-hat

# Timing
EIGHTH = 0.5    # eighth note duration in beats
BARS = 4        # how many bars to play

# Volume levels
KICK_VOL = 1.0
SNARE_VOL = 0.9
HIHAT_VOL = 0.5

# Fresh session
s = Session(tempo=120)
drums = s.new_part("Drum Kit")  # Use "Drum Kit" to get actual drum sounds


def play_drum_pattern():
    """
    Play a standard pop beat pattern for one bar (4 beats).
    
    Beat:    1   &   2   &   3   &   4   &
    Hi-hat:  x   x   x   x   x   x   x   x
    Snare:       S       S       S       S
    Kick:    K       K       K
    """
    for beat in range(4):
        play_beat(beat)


def play_beat(beat):
    """Play one beat of the pattern (two eighth notes)."""
    is_downbeat = beat in (0, 2)  # beats 1 and 3
    is_backbeat = beat in (1, 3)  # beats 2 and 4
    
    # First eighth note
    play_eighth_note(kick=is_downbeat, snare=is_backbeat)
    
    # Second eighth note (the "&")
    play_eighth_note(kick=False, snare=False)


def play_eighth_note(kick=False, snare=False):
    """Play one eighth note with optional kick and snare."""
    # Hi-hat on every eighth note
    drums.play_note(HIHAT, HIHAT_VOL, EIGHTH, blocking=False)
    
    if kick:
        drums.play_note(KICK, KICK_VOL, EIGHTH, blocking=False)
    
    if snare:
        drums.play_note(SNARE, SNARE_VOL, EIGHTH, blocking=False)
    
    # Wait for the eighth note duration
    s.wait(EIGHTH)


# Play the pattern
print(f"Playing {BARS} bars at {s.tempo} BPM...")
for bar in range(BARS):
    play_drum_pattern()

print("Done!")


Using preset Taiko Drum for Drum Kit
Playing 4 bars at 120.0 BPM...
Done!
